# **Machine Learning Part 2: Prediction Models and Model Evaluation**

Tian Lou

In this notebook, we use ML models logistic regression, decision tree, and random forest to predict IL certified claimants who will stay in the UI program for 13 weeks or longer. We evaluate each model's performance by looking at accuracy, precision at K, and recall at K. We also compare each model with baselines, i.e., randomly guessing outcomes and guessing everyone will stay for 13 weeks or longer. The data has already been cleaned and transformed to the format we need in the [machine learning data preparation notebook](./4.Machine_Learning_Data_Preparation.ipynb).

## **1. Load the Data**

As always, let's load R packages and estabilish the database connection first. Note that in this notebook, we include additional packages `caret`, `rpart`, `rpart.plot`, and `randomForest` to get functions for calculating evalution metrics and for fitting decision tree model and random forest model.

In [ ]:
# Database interaction imports
library(odbc)

# For data manipulation/visualization
library(tidyverse)

# For faster date conversions
library(lubridate)

# Classification and regression training package. For streamlining the process for creating predictive models
library(caret)

# For Decision Tree model and plot the tree
library(rpart)
library(rpart.plot)

# For Random Forest model
library(randomForest)

library(scales)

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Database = "tr_dol_eta",
                     Trusted_Connection = "True")

Let's import the data and check its top records. <font color=red> Note that you need to change the directory in read.csv() statements below. Replace ". ." with your username.</font>

In [ ]:
#Import the data that we have cleaned in the first ML notebook
df_ml <- read.csv("U:\\..\\ETA Training\\Data\\ml_data.csv")

#See the top records of the DataFrame
head(df_ml)

Take a look at the type of each column. Note that the categorical features are still in character type, such as gender and race, or integer type, such as industry code and occupation code. We need to convert them into factor type so that they can be used as dummy variables in ML models.

In [ ]:
#Convert categorical variables into factor type
df_ml$gender <- factor(df_ml$gender)
df_ml$race <- factor(df_ml$race)
df_ml$ethnicity <- factor(df_ml$ethnicity)
df_ml$disability <- factor(df_ml$disability)
df_ml$education <- factor(df_ml$education)
df_ml$naics_maj_code_rv <- factor(df_ml$naics_maj_code_rv)
df_ml$occupation_code <- factor(df_ml$occupation_code)

#### **Split the Training Set and the Testing Set**

Now we need to split our data into the training set, `df_training`, and the testing set, `df_testing`. **We will train ML models on cohort 1**, claimants who entered the UI program during the week ending March 28th and the week ending April 4th. **Then, we will validate the models on cohort 2**, claimants who entered the UI program during the week ending June 27th and the week ending July 4th.

Moreover, we only need the label and features to run a ML model. Therefore, we remove columns `ssn_id`, `cohort`, and `byr_start_week`, which are person or cohort identifiers and won't be used in our model.

In [ ]:
# The training set is the COVID-19 cohort (cohort 1)
df_training <- df_ml %>% 
    filter(cohort == 'cohort1') %>%
    select(-c(ssn_id,cohort,byr_start_week)) # remove identifiers, since we do not need them in the ML model

# The testing set is the cohort of claimants entered 13 weeks later (cohort 2)
df_testing <- df_ml %>% 
    filter(cohort == 'cohort2') %>%
    select(-c(ssn_id,cohort,byr_start_week)) # remove identifiers, since we do not need them in the ML model

## **2. Create Functions**

Some code will be used many times in this notebook, such as the code to calculate precision and recall, the code to get precision at K, and the code to compare a model's results with baselines. We can put these code in functions so that we can easily repeat these calculation processes by using one line of code.

The way to use the functions we create by ourselves is the same as how we use the functions that are already in R. We need to call the function and define the function's arguments. Then the function will return a result, such as a number or a DataFrame. Below is the list of functions we create in this notebooke. We also include instructions of how to use each of the functions and what kinds of results they return. 

1. `precision_recall(test_data, label, pscore)`: calculates precision and recall 

    After we run a ML model and use its results to predict outcomes for the testing data, we can use the `precision_recall()` function to calculate precision and recall. **This function returns a DataFrame which contains precision and recall at various k (0<k<1).** It has three arguments:
    - `test_date`: the DataFrame of the testing data
    - `label`: the name of the label column. It should be a **string**.
    - `pscore`: the name of the predicted score column. It should be a **string**. 

2. `precision_at_k(k, test_data, label, pscore)`: calculate precision at K. 

    We can use the `precision_at_k()` function to get the precision at the k% we choose. **This function returns a number.** It has four arguments:
    - `k`: the percent of population we have enough resources to intervene (e.g., help claimants get back to work)
    - `test_date`: the DataFrame of the testing data
    - `label`: the name of the label column. It should be a **string**.
    - `pscore`: the name of the predicted score column. It should be a **string**. 

3. `compare_w_baseline(k, model, test_data, label, pscore)`: compare a model's precision at k with baselines

    This function compares our ML model with two baseline models. In the first baseline models, we randomly assign labels (0 or 1) to each person. In the second baseline model, we assign 1 to every person. **This function returns a DataFrame which includes precision at k (we choose the k) of our ML model and the two baseline models.** It has five arguments: 
    - `k`: the percent of population we have enough resources to intervene (e.g., help claimants get back to work)
    - `model`: the name of the ML model. It should be a **string**
    - `test_date`: the DataFrame of the testing data
    - `label`: the name of the label column. It should be a **string**.
    - `pscore`: the name of the predicted score column. It should be a **string**. 


Now we just need to run the next three blocks of code so that they will be ready for us to use in the model evaluation.

In [ ]:
#Create a function, which returns a DataFrame containing precision and recall at K% of the population
precision_recall <- function(test_data, label, pscore) {
    
    # Get the actual label and predicted score in the testing data
    df_temp <- df_testing[, c(label, pscore)] 
    
    #Calculate Precision and Recall at K
    df_temp <- df_temp %>%
        arrange(desc(!!sym(pscore))) %>% # Sort the rows descendingly based on the predicted score
        mutate(rank = row_number()) %>% # Add rank to each row
        mutate(recall = cumsum(label == 1)/sum(label == 1),  # Calculate Recall at K
               precision = cumsum(label == 1)/rank, # Calculate precision at k
               k = rank/(nrow(test_data))) # Percent of population
    
    return(df_temp)
}

In [ ]:
#Create a function, which returns precision at K
precision_at_k <- function(k=0.1, test_data, label, pscore) {
    
    # Get the Precision-Recall at K DataFrame
    df <- precision_recall(test_data, label, pscore)
    
    # Assign a few parameters
    pct_pop <- k  # Percent of population the resource can cover
    test_pop <- nrow(df_testing) # Total number of people in the testing data
    pop_at_k <- as.integer(pct_pop * test_pop) # At K percent of the population, how many people the recourse can cover
    
    # Get precision at K% from the Precision-Recall DataFrame
    prec_at_k <- df$precision[pop_at_k]
    
    return(prec_at_k)
}

In [ ]:
# Create a function, which returns a DataFrame containing measures for baseline models 
# Baseline model 1: Randomly assign the label (0 or 1)
# Baseline model 2: Guess everyone is a slow exiter

compare_w_baseline <- function(k=0.1, model, test_data, label, pscore) {
    # Set a seed so we get consistent results
    set.seed(42)
    
    # Assign a few parameters
    pct_pop <- k  # Percent of population the resource can cover
    test_pop <- nrow(df_testing) # Total number of people in the testing data
    pop_at_k <- as.integer(pct_pop * test_pop) # At K percent of the population, how many people the recourse can cover
    
    # Get the Precision-Recall at K DataFrame
    df <- precision_recall(test_data, label, pscore)
    
    # Generate Precision-Recall at K for baseline model 1
    df_random <- df %>%
        mutate(random_score = runif(nrow(df))) %>% # Generate a row of random scores
        arrange(desc(random_score)) %>% # Sort the data by the random scores
        mutate(random_rank = row_number()) %>% # Add rank to each row
        mutate(random_recall = cumsum(label==1)/sum(label==1), # Calculate Recall at K
           random_precision = cumsum(label==1)/random_rank, # Calculate Precision at K
           random_k = random_rank/(nrow(df)))
    
    # Precision at K of the model
    model_precision_at_k <- precision_at_k(k, test_data, label, pscore)
    
    # Precision at K of baseline model 1
    random_precision_at_k <- df_random$random_precision[pop_at_k]
    
    # Precision at K of baseline model 2
    allstay_precision_at_k <- sum(test_data$label)/test_pop
    
    # Create a DataFrame which shows all measures
    df_compare_prec <- data.frame("model" = c(model, "Random", "All Slow Exiters"),
                              "precision" = c(model_precision_at_k, random_precision_at_k, allstay_precision_at_k))
    
    return(df_compare_prec)
}

## **3. Logistic Regression**

In R, we can use the function `glm()` to run a logistic regression model. The first argument, `label ~ .`, means that we want to use the column `label` as the outcome variable and use the rest of the columns as predictors. If you only want to use a few columns as predictors, such as gender and race, you need to specify these columns after `~`, i.e., `label ~ gender + race`. The second argument, `family = binomial (link = 'logit')`, defines the functional form and the error distrition. In the last argument, `data = df_training`, we indicate that we want to train the model on `df_training`. The results are saved in the object `lr_model`.

In [ ]:
# Run the logit regression with the training dataset
lr_model <- glm(label ~ ., family = binomial(link = 'logit'), data = df_training)

#Show feature importance
summary(lr_model)

In the above results, the column **Estimate** shows the importance of each feature in predicting whether a claimant will stay for 13 weeks or longer. The stars at the end of each row indicates whether a feature coefficient is significant. Note that our model automatically leaves out one category for each group of dummy variables. For example, of all the gender dummies (female, male, and unknown), female is the omitted category. 

We can see that based on our logistic regression model, REDACTED,REDACTED, REDACTED, REDACTED, REDACTED, and REDACTED are important predictors. For example, compared to REDACTED, REDACTED, REDACTED, or even REDACTED are less likely to stay in the UI program for 13 weeks or longer. For another example, claimants from REDACTED (NAICS=REDACTED) and REDACTED (NAICS=REDACTED) indusries are more likely to leave the UI program early. REDACTED claimants and claimants with REDACTED are more likely to stay in the UI program longer.

### **Model Evaluation**

Next, let's evaluate the logistic regression's performace. We will start with the confustion matrix and accuracy. Then we look at its precision at 10% of the population and compare it with the two baseline models. Finally, we create the precision and recall curve to investigate how our model perform at different thresholds and different selections of k.

#### **Accuracy**

First, let's use the function `predict()` to predict outcomes of claimants in our testing data. Inside of this function, `lr_model` is the object in which we save the logistic regression's results. `df_testing` is the testing data. `type = 'response` indicates that we want the predicted probabilities (or predicted scores). We add these predicted scores as a column `predict_score` to the DataFrame `df_testing`.

Recall that the predicted scores don't tell us the predicted outcome of a claimant. We can say that a claimant with score 0.9 is predicted to be less likely to leave the UI program than a claimant with score 0.7. But whether 0.9 implies the claimant will leave the program or not depends on our choice of the threshold. For example, if we choose a threshold of 0.8, any claimants with predicted scores greater than 0.8 will be defined as slow exiters and any claimants with predicted scores less than 0.8 will be defined as faster exiters. We save the predicted outcomes in a column `predict_label` in the DataFrame `df_testing`. 

Then we use the `confusionMatrix()` function to get the confusion matrix. This function also returns various measures, such as accuracy, sensitivity (or recall), and specificity. You can adjust the threshold based on a specific measure. For example, if you care about how accurate the predicted outcomes are, you can choose a threshold that leads to relatively high accuracy. With our current choice, threshold = 0.8, accuracy is REDACTED, which means REDACTED% of our predicted outcomes are the same as the actual outcomes. When we change the threshold to a higher value, accuracy increases. 

$$Accuracy = \frac{True Positive + True Negative}{True Positive + False Positive + True Negative + False Negative}$$

In [ ]:
# Predict the slow exiters with the coefficients of our model and the testing set
df_testing$predict_score <- predict(lr_model, df_testing, type = 'response')

# Set a threshold for the predicted score
# Assume people who get more than 0.8 predicted score will be slow exiter
threshold <- 0.8

# Add the predicted outcome to a new column in df_testing
# If predicted score is greater than the threshold, then predict label = 1, otherwise, predict label =0.
df_testing$predict_label <- ifelse(df_testing$predict_score > threshold, 1, 0) 

#Confusion matrix
confusionMatrix(factor(df_testing$predict_label), factor(df_testing$label))

#### **Precision at K**

Most of the time, we may not care about how accurate our model is in predicting both positive (slow exiter) and negative (faster exiter) outcomes. Instead, we may want to check how accurate our model is in predicting positive outcomes, which is captured by **precision**.

$$Precision = \frac{True Positive}{True Positive + False Positive}$$

Also, we may not choose the threshold directly. Instead, we may want to think about that given our resources, such as funding, time, and staff, what percentage of the population we can help. In the context of this project, we need to decide what percentage of claimants we can provide assistance to so that they can return to work faster. Suppose that we have enough resources to cover 10% of the claimants. We can use the function we created in section 2, `precision_at_k()`, to get the precision at 10% of the population. The function sorts the data descendingly based on the predicted score. The top 10% of claimants with the highest predicted score are predicted to be slow exiter (predicted_label = 1) and the rest are predicted to be fast exiters (predicted_label = 0).

In [ ]:
# Check precision at K%
k_pct <- 0.1 # Here, we are checking precision at 10% of the population, change the value to check precision at different K

lr_prec_at_10 <- precision_at_k(k_pct, df_testing, "label", "predict_score")

print(paste0("In the Logistic Regression Model, precision at ", label_percent()(k_pct), 
             " of the population is: ", round(lr_prec_at_10,5)))

The result implies that at 10% of the population, among all the claimants the logistic regression model predicts to stay in the UI program for at least 13 weeks, REDACTED% actually stayed for 13 weeks or longer. <font color=red> You can change the value assigned to `k_pct` in the first line of the code to explore precision at other percent of the population.</font>

#### **Compare our model with baselines**

How good is the logistic regression's precision at 10% of the population? Is it accurate enough or not? To answer this question, we can compare it with two baseline models. In the first baseline model, we randomly assign label (0 or 1) to each claimant in the testing data. In the second baseline model, we predict that every claimant in the testing data will not leave the UI program before the 13th week. 

The function we created in section 2, `compare_w_baseline()`, returns a DataFrame which shows percision at k percent of the population for the ML model we trained and the two baseline models. 

In [ ]:
# Compare precision at K% of the population with baseline models
df_compare <- compare_w_baseline(k=0.1, "Logistic Regression", df_testing, "label", "predict_score")

df_compare

We can see that the logistic regression model's precision is lower than the two baseline models' precision. This implies that at 10% of the population, compared to the logistic regression model, we can predict slow exiters more accurately by randomly guessing who will be slow exiters or by guessing that everyone will be slower exiters. 

The code below creates a bar plot to show the results.

In [ ]:
# Use a bar plot to show the comparision of the Logistic Regression model and the baselines (random guess or guess everyone stay)

# For easier reading, increase base font size
theme_set(theme_gray(base_size = 16))
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
options(repr.plot.width = 10, repr.plot.height = 5)

#Specify source dataset and x and y variables
lr_baseline_plot <- ggplot(df_compare, aes(x = model, y = precision)) + 
    geom_col() + #Plots bars on the graph
    geom_text(size = 5, aes(label = format(precision, digit = 3), vjust = -0.5)) + # Show values on top of the bar
    scale_y_continuous(breaks = seq(0, 1, 0.2), limits = c(0, 1)) + #Adjust the y scale to set the interval for tick marks
    labs(title = "Precision at 10% against the baseline, Logistic Regression", # Add graph title
         x = " ", y = 'Precision at 10%') + 
    theme(axis.title.x = element_text(face="bold"), #Adjust the style of X-axis label
          axis.title.y = element_text(face="bold"), #Adjust the styles of the two Y-axes labels
          axis.text.x = element_text(face = "bold", size = 16),
          plot.title = element_text(hjust = 0.5))  #Center the graph title

print(lr_baseline_plot)

#### **Precision-Recall Curve**

Another measure we often use to evaluate the performance of a ML model is recall (or sensitivity). It shows us what percentage of people with actual positive outcomes our model can capture. In the context of this project, recall tells us what percentage of slow exiters our ML model can accurately predict. 

In this section, instead of choosing a specific k (percent of population), we use the function we created in section 2, `precision_recall()`, to get precision and recall at various k (0 < k < 1). Then we plot their values on a line chart so that it is easier for us to see how precision and recall change with our choice of k.

$$Precision = \frac{True Positive}{True Positive + False Positive}$$

$$Recall = \frac{True Positive}{True Positive + False Negative}$$

In [ ]:
# Get Precision-Recall at K DataFrame
df_measure_at_k <- precision_recall(df_testing, "label", "predict_score")

# See the top records of the DataFrame
head(df_measure_at_k)

In [ ]:
# For easier reading, increase base font size
theme_set(theme_gray(base_size = 16))
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
options(repr.plot.width = 10, repr.plot.height = 5)

#Create the precision-recall curve
lr_pr_curve <- ggplot(df_measure_at_k, aes(x=k)) + # Plot percent of population (k) on the x-axis
    geom_line(aes(y=precision), color = 'blue') + # Add the precision curve
    geom_line(aes(y=recall), color = 'red') + # Add the recall curve
    scale_y_continuous(       # We need to create a dual-axis graph, so we need to define two y axes
        name = "Precision",     # Label of the first axis
        sec.axis = sec_axis(~.*1,name="Recall"), # Add a second axis and specify its label
        breaks = seq(0, 1, 0.1)) +  # Adjust the tick mark on Y-axis
    scale_x_continuous(breaks = seq(0, 1, 0.2)) + # Adjust the tick mark on X-axis
    labs(title = "Precision-Recall Curve, Logistic Regression", # Add graph title
         x = "Percent of Population") + # Add X-axis label
    theme(axis.title.x = element_text(face="bold"), #Adjust the style of X-axis label
          axis.title.y.left = element_text(face="bold", color="blue"), #Adjust the styles of the two Y-axes labels
          axis.title.y.right = element_text(face = 'bold', color = 'red'),
          plot.title = element_text(hjust = 0.5))  #Center the graph title

#Display the graph that we just created
print(lr_pr_curve)

In the above graph, the blue line represents precision. We can see that it stays relatively constant when we increase the choice of k. The red line shows recall. Its values increases as we increase the choice of k. At the 10% of the population, both precision and recall are low. 

This graph is somewhat different from what we have seen in the class. The precision curve should start at a high point, because when k is low, only claimants with the highest predicted scores will be defined as slow exiters. When we select a higher percent of population, we also relax the threshold and predict claimants with relatively low predicted score to be slow exiters. Therefore, the precision should decrease. 

This implies that we may either need to choose another model or need to investigate our data. For example, our current features may not be enough to predict claimants' exit decisions. We can add other data sources and create more features. Also, we may need to think about whether the training data we use is representative enough to train a ML model.

## **4. Decision Tree**

In this section, we run another ML model, decision tree. The function we use is `rpart()`. Similar as before, the first argument, `label ~ . `, defines the formula of the model. The second argument, `method = 'class'` means that we want to use the classification tree method, because our label is binary. In the last argument, we indicate that we want to train the model on data in `df_training`. We save the results in the object `dt_model`.

In [ ]:
# Run the decision tree model
dt_model <- rpart(label ~ ., method = 'class', data = df_training)

# Print results
printcp(dt_model)

In the above result, we can see that the variables the decision tree model actually used to predict outcomes are average total pay, industry, occuaption, and 2019 earnings. This is somewhat consistent with what we have seen in the logistic regression model. 

Next, we use the function `prp()` to print out the tree and see how the model decides whether a claimant will become a slow exiter. You can adjust the parameters of the `prp()` function to change the display of the tree. `type` decides the overall style of the tree graph. `extra` determines the information shown at each node. Here, we choose to show percentage of observations. We also use `main` to add a title to this tree graph.[<sup>1</sup>](#fn1) <a id = "13"> </a>

In [ ]:
# Print the tree
prp(dt_model, # Your decision tree model
    type = 0, # type of trees
    extra = 100, # what information to show in each node
    main = "Decision Tree Model") # Add a title to your decision tree graph

How well does the decision tree model perform in predicting slow exiters? Next, let's use the model to predict scores for claimants in the testing data and save the predicted scores in column `df_predict_score`. Then, we calculate the precision and recall and create the precision-recall curve for the decision tree model.

In [ ]:
# Get the Decision Tree model predicted score and save it in a column in the testing DataFrame
df_testing$dt_predict_score <- predict(dt_model, df_testing, type = 'prob')[,2]

# Get the Decision Tree model Precision-Recall at K% DataFrame
df_dt_measure_at_k <- precision_recall(df_testing, "label", "dt_predict_score")

In [ ]:
# For easier reading, increase base font size
theme_set(theme_gray(base_size = 16))
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
options(repr.plot.width = 10, repr.plot.height = 5)

#Create the precision-recall curve
dt_pr_curve <- ggplot(df_dt_measure_at_k, aes(x=k)) + 
    geom_line(aes(y=precision), color = 'blue') + # Add the precision curve
    geom_line(aes(y=recall), color = 'red') + # Add the recall curve
    scale_y_continuous(       # We need to create a dual-axis graph, so we need to define two y axes
        name = "Precision",     # Label of the first axis
        sec.axis = sec_axis(~.*1,name="Recall"), # Add a second axis and specify its label
        breaks = seq(0, 1, 0.1)) +  # Adjust the tick mark on Y-axis
    scale_x_continuous(breaks = seq(0, 1, 0.2)) + # Adjust the tick mark on X-axis
    labs(title = "Precision-Recall Curve, Decision Tree", # Add graph title
         x = "Percent of Population") + # Add X-axis label
    theme(axis.title.x = element_text(face="bold"), #Adjust the style of X-axis label
          axis.title.y.left = element_text(face="bold", color="blue"), #Adjust the styles of the two Y-axes labels
          axis.title.y.right = element_text(face = 'bold', color = 'red'),
          plot.title = element_text(hjust = 0.5))  #Center the graph title

#Display the graph that we just created
print(dt_pr_curve)

We can see that the precision-recall curve for the decision tree model looks similar to the curve for the logistic regression model. Again, both precision and recall are low at 10% of the population. The decision tree model is also not very accurate in terms of predicting slow exiters.

## **5. Compare Multiple Models**

In this section, we provide you code to compare evaluation metrics for several ML models all at once. First, we "refresh" the training data and the testing data to remove columns we created in the previous two sections, such as the predicted scores. This way, we don't accidentally mix them with the results we get in this section.

In [ ]:
# The training set is the COVID-19 cohort (cohort 1)
df_training <- df_ml %>% 
    filter(cohort == 'cohort1') %>%
    select(-c(ssn_id,cohort,byr_start_week)) # remove identifiers, since we do not need them in the ML model

# The testing set is the cohort of claimants entered 13 weeks later (cohort 2)
df_testing <- df_ml %>% 
    filter(cohort == 'cohort2') %>%
    select(-c(ssn_id,cohort,byr_start_week)) # remove identifiers, since we do not need them in the ML model

Next, we need to define a few parameters and the DataFrame we use to save the results. `model_list` is a string list which shows all the models we are going to run. In addition to the logistic regression and the decision tree models, we also include the random forest model. You can expand the list if you want to include other ML models. `k` is the percent of population we are able to intervene with available resources. We also create a DataFrame, `df_compare_models`, to store the model names, accuracy, precision at k, and recall at k.

In [ ]:
#Create a list to include all the models we want to compare
#LR: Logistic Regresion
#DT: Decision Tree
#RF: Random Forest
model_list <- c("LR", "DT", "RF")

#Define percent of population the resource can cover
k <- 0.1
pct_pop <- k  # Percent of population the resource can cover
test_pop <- nrow(df_testing) # Total number of people in the testing data
pop_at_k <- as.integer(pct_pop * test_pop) # At K percent of the population, how many people the resourse can cover

#Define an empty DataFrame to save our results
n <- length(model_list) #Number of rows of the DataFrame
df_compare_models <- data.frame(Model = model_list, accuracy = double(n), 
                                precision_at_k = double(n), recall_at_k = double(n)) # Define the columns of the DataFrame

Finally, we use a loop to run all three models and calculate evaluation metrics for each model. The only new model we include in this loop is random forest. We use the function `randomForest` to run the model. Again, we need to define the formula, the data we train the model on, and the method (classification). `ntree` determines the number of trees to grow. `mtry` is the number of variables randomly sampled as candidates at each split. `importance = TRUE` means we want the importance of predictors to be assessed.[<sup>2</sup>](#fn1) <a id = "14"> </a>

If you would like to include other models, add new `if` statements inside of the loop. Model results should be saved in the object `fit` and predicted scores should be saved in the column `predict_score` of the DataFrame `df_testing`.

In [ ]:
for (model in model_list) {
    
    # Logististic Regression Model
    if (model=="LR") {
        fit <- glm(label ~ ., family = binomial(link = 'logit'), data = df_training) # Fit the model
        df_testing$predict_score <- predict(fit, df_testing, type = 'response') # Predict scores
    }
    
    #Decision Tree Model
    if (model=="DT") {
        fit <- rpart(label ~ ., method = 'class', data = df_training) # Fit the model
        df_testing$predict_score <- predict(fit, df_testing, type = 'prob')[,2] # Predict scores
    }
    
    #Random Forest Model
    if (model == "RF"){
        fit <- randomForest(label ~ ., data = df_training, type = 'class', ntree = 500, mtry = 6, importance = TRUE) # Fit the model
        df_testing$predict_score <- predict(fit, df_testing) # Predict scores
    }
    
    # Get Precision-Recall DataFrame
    df_prec_rec <- precision_recall(df_testing, "label", "predict_score")
    
    #Calculate accuracy
    threshold <- df_prec_rec$predict_score[pop_at_k] # Get the predicted score at K%
    df_testing$predict_label <- ifelse(df_testing$predict_score > threshold, 1, 0) # Predict the label, if > threshold, then 1; if < threshold, then 0
    df_testing <- df_testing %>% mutate(accurate = 1*(label == predict_label)) # Generate an indicate of whether the prediction is correct
    acc = (sum(df_testing$accurate)/nrow(df_testing)) # Calculate accuracy
    df_compare_models$accuracy <- ifelse(df_compare_models$Model == model,acc,df_compare_models$accuracy) # Save accuracy to the DataFrame
    
    #Calculate precision and save it in the df_compare_models DataFrame
    prec_at_k <- precision_at_k(k, df_testing, "label", "predict_score")
    df_compare_models$precision_at_k <- ifelse(df_compare_models$Model == model, prec_at_k, df_compare_models$precision_at_k)
    
    #Calculate Recall and save it in the df_compare_models DataFrame
    rec_at_k <- df_prec_rec$recall[pop_at_k]
    df_compare_models$recall_at_k <- ifelse(df_compare_models$Model == model, rec_at_k, df_compare_models$recall_at_k)
    
}

After we run the code in the above cell, we can check the results by looking at `df_compare_model`. We can see that at the 10% of the population, the REDACTED model is the most accurate in predicting both slow exiters and faster exiters (i.e., has the highest accuracy). However, the REDACTED is better at predicting slow exiters and can capture more slow exiters than the other two models.

In [ ]:
df_compare_models

## **Footnotes:**
<span id="fn1"> 1. For more information, see <a href='http://www.milbo.org/rpart-plot/prp.pdf'>plotting rpart trees with the rpart.plot package</a>. </span>  

[[Go back]](#13)

<span id="fn2"> 2. For more information, see <a href='https://cran.r-project.org/web/packages/randomForest/randomForest.pdf'>Breiman and Cutler's Random Forests for Classification and
Regression</a>. </span>  

[[Go back]](#14)

#### 